In [1]:
# Import packages
import numpy as np
import pandas as pd
import os
from bs4 import BeautifulSoup
import requests
from datetime import date
import re
import time

In [41]:
# Set wd
os.chdir('C:\\Users\\jgjos\\Documents\\python_resources\\personal\\personal_projects\\house_prices_ireland')

In [42]:
# Get wd
os.getcwd()

'C:\\Users\\jgjos\\Documents\\python_resources\\personal\\personal_projects\\house_prices_ireland'

In [33]:
# Create list of counties
roi_counties = ["Carlow","Cavan","Clare","Cork",
                "Donegal","Dublin",
                "Galway",
                "Kerry","Kildare","Kilkenny",
                "Laois","Leitrim","Limerick","Longford","Louth",
                "Mayo","Meath","Monaghan",
                "Offaly",
                "Roscommon",
                "Sligo",
                "Tipperary",
                "Waterford","Westmeath","Wexford","Wicklow"]

In [3]:
# Get dates
# Oldest year on PPR website
start_year = 2010
# Current dates
current_date = date.today()
# Current year
current_year = current_date.year
# Current month
current_month = current_date.month
# Current day
current_day = current_date.day

In [35]:
# Get list of possible years
years = []
for i in range(start_year,current_year+1):
    years.append(i)
# Get rid of current year (will add this part later)
years=years[0:-1]

In [6]:
# Get months
months = []
for i in range(1,13):
    months.append(i)

In [7]:
# Get list of months with zeros in front of some numbers. This is needed for the string
months_with_zeros = ['01','02','03','04','05','06','07','08','09','10','11','12']

In [8]:
# Get common strings from url
# https://www.propertypriceregister.ie/
common_string_1 = "https://www.propertypriceregister.ie/Website/npsra/PPR/npsra-ppr.nsf/PPR-By-Date&Start=1&Query=%5Bdt_execution_date%5D%3E=01/"
common_string_2 = "/"
common_string_3 = "%20AND%20%5Bdt_execution_date%5D%3C01/"
common_string_4 = "/"
common_string_5 = "%20AND%20%5Bdc_county%5D="
common_string_6 = "&County="
common_string_7 = "&Year="
common_string_8 = "&StartMonth="
common_string_9 = "&EndMonth="
common_string_10 = "&Address="

In [9]:
# Create empty arrays to loop through
url_list = []
county_array = []

# Loop for each county
for county_i in roi_counties:
    # Loop for each year
    for years_index in range(len(years)):
        # Loop for each month
        for months_index in range(len(months)):
            
            # Different for Dec as next month is 01 not 13
            if months[months_index] == 12:
                
                # Create url
                url_i = common_string_1+str(months_with_zeros[months_index])+common_string_2+str(years[years_index])+common_string_3+'1'+common_string_4+str(years[years_index]+1)+common_string_5+county_i+common_string_6+county_i+common_string_7+str(years[years_index])+common_string_8+str(months_with_zeros[months_index])+common_string_9+str(months_with_zeros[months_index])+common_string_10
                # Append to empty arrays
                url_list.append(url_i)
                county_array.append(county_i)
                        
            else:

                # Create url
                url_i = common_string_1+str(months_with_zeros[months_index])+common_string_2+str(years[years_index])+common_string_3+str(months[months_index]+1)+common_string_4+str(years[years_index])+common_string_5+county_i+common_string_6+county_i+common_string_7+str(years[years_index])+common_string_8+str(months_with_zeros[months_index])+common_string_9+str(months_with_zeros[months_index])+common_string_10
                # Append to empty arrays
                url_list.append(url_i)
                county_array.append(county_i)
                          

In [10]:
# Get start time
start = time.time()

# Create empty array
array_i=[]
array_x=[]

# Loop through url list
for url_i in url_list:
    
    # Filter HTML code for relevant data
    page = requests.get(url_i,verify=False).text
    soup = BeautifulSoup(page, 'html.parser')
    html_part_1=soup.prettify()
    html_part_2=re.split("SearchResults = ",html_part_1)
    html_part_3=html_part_2[1]
    html_part_4=re.split("DD/MM/YYYY",html_part_3)
    html_part_5=html_part_4[0][1:-29]
    html_part_6=re.split("\[",html_part_5)
    html_part_7=html_part_6[1:]
    
    # Loop through all the sales that month
    for i in range(len(html_part_7)):
    
        # Date
        sale_date_i=html_part_7[i][1:11]

        # Price
        full_stop_location=re.search("\.", html_part_7[i]).span()[0]
        price_i=html_part_7[i][15:full_stop_location+3]

        # Address
        address_location_1=re.search(">", html_part_7[i]).span()[0]+1
        address_i=html_part_7[i][address_location_1:-7]

        # Add data to array
        array_i.append((sale_date_i,price_i,address_i))
    
    # Used for adding county column to final df
    array_x.append(len(html_part_7))

# Get end time
end = time.time()
# Time Elapsed (mins)
round((end-start)/60)

C:\Users\jgjos\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.propertypriceregister.ie'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\jgjos\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.propertypriceregister.ie'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\jgjos\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.propertypriceregister.ie'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\jgjos\anacond

79

In [12]:
# Create dataframe
df_final=pd.DataFrame(array_i,columns=("Date (DD/MM/YYYY)","Price (€)","Address"))

In [19]:
# Create empty array
new_county_array=[]

# Loop through all the urls
for i in range(len(county_array)):
    
    # Create empty array
    empty_array=[]
    freq=array_x[i]
    # Get county corresponding to url i
    empty_array.append(county_array[i])
    # Replicate so that it accounts for all sales for that particular url/month
    full_array=empty_array*freq
    new_county_array.append(full_array)
    

In [23]:
# Create empty array
county_array_merged = []

# Loop through all lists in new_county_array list
for sublist in new_county_array:
    # Create one final list with items from all sublists
    for item in sublist:
        county_array_merged.append(item)

In [30]:
# Create county column
df_final['County']=county_array_merged

In [32]:
# Output final dataframe
df_final.tail()

,Date,Price,Address,County
456988,01/12/2020,"411,000.00","4 LAKESIDE DOWNS, BLESSINGTON, WICKLOW",Wicklow
456989,01/12/2020,"535,000.00","8 CHURCH AVE, EDEN GATE, DELGANY",Wicklow
456990,01/12/2020,"325,000.00","86 CHARLESLAND PARK, THE GLEN, GREYSTONES",Wicklow
456991,01/12/2020,"1,080,000.00","MILLICENT HILL, KILCRONEY LANE, BRAY",Wicklow
456992,01/12/2020,"600,000.00","SINGLETON, SEA ROAD, ARKLO",Wicklow


In [43]:
df_final.to_excel("House Prices.xlsx") 